In [ ]:
#importing relevant lybraries
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 500) #in order to better look at results

#importing data
data=pd.read_csv('customer_data_sample.csv')


In [ ]:
data[data['age']<1]

In [ ]:
#Step0: looking at the data
data.head()
data.tail()

In [ ]:
data.branch.unique()
data.gender.unique()

In [ ]:
#converting gender and branch into a numerical value:
female=[]
for g in data.gender:
    if g=='female': female.append(1)
    else: female.append(0)
data['female']=female

branch_n=[]
for b in data.branch:
    if b=='Helsinki': branch_n.append(0)
    elif b=='Tampere': branch_n.append(1)
    elif b=='Turku': branch_n.append(2)
    else: branch_n.append(None)

data['branch_n']=branch_n

In [ ]:
data.tail()

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data.age.unique()
#Something is wrong with the Ages

From these few observation i think it is needed some cleaning of the variable "age".

According to danish law, "The Danish government recently passed legislation that prohibits the collection and processing of personal data belonging to children below the age of 13" (https://www.jurist.org/news/2023/06/denmark-to-raise-age-at-which-tech-companies-can-collect-minors-data/). 
Among them there are 7 whose values is lower than 1 (e.g. age=0.42) that may be a typing mistake and few more where the age of the customer would lead me to think about a wrong informatoin e.g. age=2 or age=4. Those have the highest frequencies - 10 each.

In [ ]:
data_age=data[data['age']<13]
data_age.groupby('age')['customer_id'].count()

In [ ]:
sum(data_age.groupby('age')['customer_id'].count())

OBSERVATIONS:
No additional information on the ages at this point.

We have overall 38% of conversion rate and the customers have less than 30 years. On average one person has 2 customers (see 'Related customer' variable). majority of customers seems to have 0 family_size. on average the initial_fee_level is 64.4

In [ ]:
data.initial_fee_level.unique()[:10]
#initial fee level seems to be quite volatile as variable. there is no additional information on the trace. But it looks a bit suspicious.

In [ ]:
#checking nulls
data.isnull().sum()

In [ ]:
#Cleaning data form the "suspicious ages"
data_clean=data#[data['age']>13]
#dropping duplicates
data_clean=data_clean.drop_duplicates()
#dropping empty cells
data_clean=data_clean.dropna()
#rounding the age into an integer
age_r=[int(a) for a in data_clean.age]
data_clean['age_r']=age_r
#cleaning the index
data_clean.reset_index(inplace=True, drop=True)


In [ ]:
#checking new data
data_clean.nunique()

In [ ]:
data_clean.describe()

In [ ]:
data_clean.groupby('credit_account_id')['customer_id'].count()
data.groupby('credit_account_id')['customer_id'].count()

credit_account_id_d=[]
for c in data_clean['credit_account_id']:
    if c=='9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6c9bc9d493a23be9de0':
        credit_account_id_d.append(1)
    else:
        credit_account_id_d.append(0)
data_clean['credit_account_id_dummy']=credit_account_id_d

OBSERVATIONS:
Changes seems not to affect deeply the averages of the variables (excluding age for obvious reasons)

Dropping the nempty cells and the duplicates (and cleaning for the age<13) we lost 250 observation (around 28% of the data). A deeper knowledge of the nature of the data itself could lead me to do not delete and try to save some of it. Given the current situation i prefer continuing with the analysis of the 641 "clean" data. 

9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6c9bc9d493a23be9de0  had  687 observation. after cleaning it went down to 469. (218 out of the lost 250). I will create a dummy variable where 1 indicated the id above with the majority of observations and 0 all the others ids.

Before proceeding I will make a quick check of the other variables in the dataset

In [ ]:
#Checking variables
data_clean.groupby('family_size')['customer_id'].count()
data_clean.groupby('customer_id')['customer_id'].count()
data_clean.groupby('converted')['customer_id'].count()
data_clean.groupby('customer_segment')['customer_id'].count()
data_clean.groupby('related_customers')['customer_id'].count()
data_clean.groupby('family_size')['customer_id'].count()
data_clean.groupby('initial_fee_level')['customer_id'].count()
data_clean.groupby('female')['customer_id'].count()
data_clean.groupby('branch_n')['customer_id'].count()
data_clean.groupby('credit_account_id_dummy')['customer_id'].count()
print('ok')


Observations:
vast majority of the customers are in segment 13 (306). seg 11=180 and seg 12=155.

Most of the customer has 0 People related to them (not sure about the meaning of this varible) 0=450, 1=162, 2=19, 3=5, 4=3, 5=2

Mos of family members are 0. 0=515, 1=74, 2=37, 3=5, 4=4, 5=5, 6=1

initial fee level seems to be quite volatile as variable. there is no additional information on the trace. But it looks a bit suspicious.

Most of the customer is male 416 vs 225

Most of the customers are in Helsinki=499. Tampere=118, Turku=24 


WE CAN NOW START OUR ANALYSIS

having the information above in mind


In [ ]:
import seaborn as sns
sns.set_style("whitegrid")
sns.pairplot(data_clean)

In [ ]:
sns.countplot(data=data_clean, x='converted')
plt.rcParams['figure.figsize'] = [10,3]

In [ ]:
sns.countplot(data=data_clean, x='age_r', hue='converted')
plt.rcParams['figure.figsize'] = [100,7]

In [ ]:
sns.countplot(data=data_clean, x='initial_fee_level', color='blue', linewidth=1)
plt.rcParams['figure.figsize'] = [100,20]

In [ ]:
sns.countplot(data=data_clean, x='initial_fee_level', hue='converted')
plt.rcParams['figure.figsize'] = [100,20]

In [ ]:
sns.countplot(data=data_clean, x='customer_segment', color='blue')
plt.rcParams['figure.figsize'] = [10,3]

In [ ]:
sns.countplot(data=data_clean, x='customer_segment',hue='converted')

In [ ]:
sns.countplot(data=data_clean, x='branch', color='blue')


In [ ]:
sns.countplot(data=data_clean, x='branch',hue='converted')


In [ ]:
sns.countplot(data=data_clean, x='gender', color='blue')
#plt.xticks(ticks=[0, 1], labels=['Male', 'Female'])

In [ ]:
sns.countplot(data=data_clean, x='gender',hue='converted')

In [ ]:
sns.countplot(data=data_clean, x='related_customers',color='blue')

In [ ]:
sns.countplot(data=data_clean, x='related_customers',hue='converted')

In [ ]:
sns.countplot(data=data_clean, x='family_size',color='blue')

In [ ]:
sns.countplot(data=data_clean, x='family_size',hue='converted')

In [ ]:
sns.countplot(data=data_clean, x='credit_account_id_dummy',color='blue')

In [ ]:
sns.countplot(data=data_clean, x='credit_account_id_dummy',hue='converted')

In [ ]:
# Graphing correlation between columns
sns.heatmap(data_clean.corr(), annot=True, cmap='coolwarm', linewidths=.5)
plt.rcParams['figure.figsize'] = [30, 10]


In [ ]:
sns.countplot(data=data_clean, x='credit_account_id_dummy',hue='customer_segment')

Observations:
from the correlation table we can conclude that there is a mmoderate correlation between the gender and the converted variable. specifically, female seems to increase the probability to convert compared to male.

A mild positive correlation seems to exists between te variable converted and the variable "initial_fee_level". Nameling increasing the initial fee level, increase the probability to convert.

A mild negative correlation exist with the customer segment. Segment 11 seems to increase the probability to convert customers compared to segment 12 and expecially 13. I have no further information on the customer segment, therefore it is hard to draw conclusions with this.

A strong correlation exists between the Customer segment and the credit account they have. it looks like that segment 11 is almost never in the residual category of the credit account_ids. just the opposite for the other two segments.
This imply a mild negative correlation between the credit_account_id and the converted variable.

In [ ]:
# Graphing correlation between columns
sns.heatmap(data_clean[['converted','female','initial_fee_level','customer_segment','credit_account_id_dummy' ]].corr(), annot=True, cmap='coolwarm', linewidths=.5)
plt.rcParams['figure.figsize'] = [20, 7]
plt.show()

In [ ]:
sns.countplot(data=data_clean, x='gender',hue='converted')

In [ ]:
sns.countplot(data=data_clean, x='customer_segment',hue='converted')

In [ ]:
sns.countplot(data=data_clean, x='credit_account_id_dummy',hue='converted')

"What are the most important factors for predicting whether a customer has converted or not?"

In order to find the relevant factors I need to look at the nature of the problem and it´s component.
we are trying to understand how much the probability of an event to happen (converted or not) given the behaviour of the other variables (independent variables).

The nature of the dependent variable is categorical (dicotomical) and the nature of the dependent variables is different for each variable (categorical and/or interval (continuos), since we are dealing with different variables.) 

litterature* suggests to use a multiple logistic regression to identify the variable that best explain the probability to convert.

*https://stats.oarc.ucla.edu/other/mult-pkg/whatstat/; https://stats.oarc.ucla.edu/spss/whatstat/what-statistical-analysis-should-i-usestatistical-analyses-using-spss/#logistic


In [ ]:
#make the dataset ready to the analysis
df=data_clean[['customer_id','converted','female','age_r','initial_fee_level','credit_account_id_dummy']]
df.head()

In [ ]:
variables=['customer_segment','related_customers','family_size','branch']
for v in variables:
    temp=pd.get_dummies(data_clean[v], prefix=v)
    for c in temp.columns:
        df[c]=temp[c]

In [ ]:
df.columns

In [ ]:
#Logistic regression
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Split data into features (X) and target variable (y)
X = df.drop('converted', axis=1)  # Features
y = df['converted']  # Target variable

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize logistic regression model
logreg = LogisticRegression()

# Fit the model on the training data
logreg.fit(X_train, y_train)

# Predict on the test data
y_pred = logreg.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Print classification report
print(classification_report(y_test, y_pred))

# Get coefficients
coefficients = logreg.coef_[0]

# Map coefficients to their corresponding feature names
feature_coefficients = pd.DataFrame({'Feature': X.columns, 'Coefficient': coefficients})

# Sort the features by their coefficients
feature_coefficients = feature_coefficients.reindex(feature_coefficients['Coefficient'].abs().sort_values(ascending=False).index)

print("Feature Coefficients:")
print(feature_coefficients)

In [ ]:
#alternative method using statsmodel
import statsmodels.api as sm

# Split data into features (X) and target variable (y)
X_a = df.drop('converted', axis=1)  # Features
y_a = df['converted']  # Target variable

# Add intercept term to the independent variables
X_a = sm.add_constant(X_a)

# Fit logistic regression model
logit_model = sm.Logit(y, X)

# Obtain the results of the logistic regression
logit_result = logit_model.fit()

# Print summary of the logistic regression results
print(logit_result.summary())

In [ ]:
['converted', 
'female', 'age_r', 'initial_fee_level','credit_account_id_dummy', 
'customer_segment_11', 'customer_segment_12','customer_segment_13', 
'related_customers_0', 'related_customers_1','related_customers_2', 'related_customers_3', 'related_customers_4','related_customers_5', 
'family_size_0', 'family_size_1','family_size_2', 'family_size_3', 'family_size_4', 'family_size_5', 'family_size_6', 
'branch_Helsinki', 'branch_Tampere', 'branch_Turku']

In [ ]:
df3=df[[ 'converted', 'female', 
         'related_customers_0',#'family_size_1','age_r',#  
         'customer_segment_11',#'credit_account_id_dummy',#'initial_fee_level',#'credit_account_id_dummy',
        'branch_Helsinki']]

#alternative method using statsmodel
import statsmodels.api as sm
import numpy as np

# Split data into features (X) and target variable (y)
X_a = df3.drop('converted', axis=1)  # Features
y_a = df3['converted']  # Target variable

# Fit logistic regression model
logit_model = sm.Logit(y_a, X_a)

# Obtain the results of the logistic regression
logit_result = logit_model.fit()

# Print summary of the logistic regression results
print(logit_result.summary())

odds_ratios = np.exp(logit_result.params)
conf_intervals = np.exp(logit_result.conf_int())

# Create a DataFrame to display the odds ratios and their confidence intervals
odds_ratios_df = pd.DataFrame({'Odds Ratio': odds_ratios, '95% CI Lower': conf_intervals[0], '95% CI Upper': conf_intervals[1]})
print(odds_ratios_df)

Results:

gender and age are significant (female have almost 12 time higher effect than male). the younger the better (2% for each year) 

initial fee level has a very small, but  positive and significant effect on the conversion. it increases a lot in segment 11 (and therefoe when dumy credit account is 0)

Customer segment 11 seems to increase the convertion rate significantly (+28% on seg_12 and + 74% on seg_13). by itself it increase by 1.8 times

related customer 0 is decrease the probability to convert compared to related_customer_5 (-47%). none of the other is significant

Family size 2 seems to have a mildly significant increase compared to family size 0. (1.8 times)

Helsinki decrease the probability of conversion by 50% compared to Tampere and Turku. 


In [ ]:
data_clean.groupby('customer_segment')['initial_fee_level'].mean()